<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/Python_Code_to_Move_PST_Emails_to_Inbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime
import json
import os

class PSTtoOSTMigrator:
    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'total_attempted': 0,
            'total_successful': 0,
            'total_failed': 0,
            'failed_items': [],
            'folder_summary': {}
        }

    def setup_logging(self):
        """Setup comprehensive logging"""
        # Ensure the logs directory exists
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'pst_migration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        # Also log to console
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)
        logging.info(f"Logging to: {log_filename}")

    def validate_move(self, original_item, target_folder, timeout=30):
        """
        Validate that an item was successfully moved by checking if it exists in target.
        We can't reliably check if it's *no longer* in source as the source might still hold a reference
        until refreshed or Outlook syncs, and the original item object might become invalid.
        """
        subject = getattr(original_item, 'Subject', 'N/A')
        entry_id = getattr(original_item, 'EntryID', None)
        sent_on = getattr(original_item, 'SentOn', None)
        sender_name = getattr(original_item, 'SenderName', 'N/A')
        validation_passed = False

        logging.debug(f"Attempting to validate move for: {subject[:50]}...")

        start_time = time.time()
        while time.time() - start_time < timeout:
            try:
                # Search for the item in target folder by EntryID or subject/sent time
                # Using Restrict for efficiency, especially with large folders
                restriction = ""
                if entry_id:
                    # Searching by EntryID is most reliable but requires iterating, or using AdvancedSearch (more complex)
                    # For simple GetFirst/GetNext on Items, a direct search isn't trivial for EntryID
                    # Let's use Subject and SentOn as a practical alternative for validation
                    # Note: Outlook's MAPI won't return EntryID for moved items directly sometimes,
                    # so matching properties is safer.
                    pass # We'll build the restriction below

                if subject and sent_on:
                    # Format SentOn for Restriction. SentOn is a datetime object.
                    # Need to convert it to a string format Outlook understands for restriction
                    # Example: "[SentOn] = '6/15/2023 10:30:00 AM'"
                    # Be careful with locale and timezones. Using ISO format might be better if Outlook supports it.
                    # Outlook typically prefers 'MM/DD/YYYY HH:MM:SS AM/PM' or 'YYYY-MM-DD HH:MM:SS'
                    # Let's try to match a common format.
                    # This can be tricky due to timezone differences or Outlook internal representation.
                    # For simplicity, we'll try to match Subject and SenderName if SentOn causes issues.
                    # For robust comparison, we'd ideally compare each item's properties after fetching.

                    # Let's use a simpler restriction for initial check
                    # The replace is for handling apostrophes in subjects for MAPI queries
                    escaped_subject = subject.replace("'", "''")
                    # Converting SentOn to string format for comparison, if available.
                    # This part is notoriously tricky due to time zones and date formatting.
                    # A robust approach would involve fetching all items with the subject and then comparing dates.

                    # Simpler approach: check only subject if date comparison is too fragile
                    restriction = f"[Subject] = '{escaped_subject}'"

                    found_items = target_folder.Items.Restrict(restriction)

                    if found_items.Count > 0:
                        # Iterate through found items to get a better match
                        for found_item in found_items:
                            # Compare key properties. Date comparison needs care.
                            # Compare dates as strings or by converting to a common timezone if possible.
                            # For simplicity, we compare subject and sender name for now, assuming SentOn is less critical for *existence* validation.
                            if (getattr(found_item, 'Subject', 'N/A') == subject and
                                getattr(found_item, 'SenderName', 'N/A') == sender_name):
                                # If SentOn is very important, add robust date comparison here
                                # e.g., if original_item.SentOn and found_item.SentOn and \
                                #    abs((original_item.SentOn - found_item.SentOn).total_seconds()) < 60:
                                validation_passed = True
                                logging.info(f"✓ Validation PASSED: {subject[:50]}...")
                                break
                        if validation_passed:
                            break # Break from while loop if validation passed

            except Exception as e:
                logging.debug(f"Validation attempt failed for {subject[:50]}...: {e}")

            time.sleep(1)  # Wait before retry

        if not validation_passed:
            logging.error(f"✗ Validation FAILED after {timeout} seconds for: {subject[:50]}...")

        return validation_passed

    def get_item_signature(self, item):
        """Create a unique signature for reporting/logging"""
        try:
            return {
                'subject': getattr(item, 'Subject', 'N/A'),
                'sent_on': str(getattr(item, 'SentOn', None)) if getattr(item, 'SentOn', None) else None,
                'sender': getattr(item, 'SenderName', 'N/A'),
                'entry_id': getattr(item, 'EntryID', 'N/A'),
                'size': getattr(item, 'Size', 'N/A')
            }
        except Exception as e:
            return {'subject': 'Unknown', 'error': f'Could not get item properties: {e}'}

    def process_folder(self, source_folder, target_inbox_folder, folder_path=""):
        """
        Process all items in a folder and move them to the specified target_inbox_folder.
        This version ensures all emails go directly to the Inbox.
        """
        success_count = 0
        error_count = 0

        current_folder_path = f"{folder_path}/{source_folder.Name}" if folder_path else source_folder.Name

        logging.info(f"Processing folder: {current_folder_path}")

        try:
            items = source_folder.Items
            item_count = items.Count
            logging.info(f"Found {item_count} items in '{current_folder_path}' to process.")

            # Process items in reverse to avoid index issues when items are moved
            for i in range(item_count, 0, -1):
                try:
                    item = items[i]
                    # olMail item class is 43
                    if item.Class == 43:
                        self.migration_report['total_attempted'] += 1

                        original_signature = self.get_item_signature(item)
                        logging.info(f"Attempting to move item: {original_signature['subject'][:50]}...")

                        try:
                            item.Move(target_inbox_folder)

                            # Validate the move
                            if self.validate_move(item, target_inbox_folder):
                                success_count += 1
                                self.migration_report['total_successful'] += 1
                                logging.info(f"Successfully moved and validated: {original_signature['subject'][:50]}...")
                            else:
                                error_count += 1
                                self.migration_report['total_failed'] += 1
                                self.migration_report['failed_items'].append({
                                    'folder': current_folder_path,
                                    'subject': original_signature['subject'],
                                    'error': 'Validation failed after move (item not found in target)',
                                    'original_signature': original_signature
                                })
                                logging.error(f"Move validation failed for: {original_signature['subject']}")

                        except Exception as move_error:
                            error_count += 1
                            self.migration_report['total_failed'] += 1
                            self.migration_report['failed_items'].append({
                                'folder': current_folder_path,
                                'subject': original_signature['subject'],
                                'error': str(move_error),
                                'original_signature': original_signature
                            })
                            logging.error(f"Move failed for '{original_signature['subject']}': {move_error}", exc_info=True)

                except Exception as item_error:
                    error_count += 1
                    logging.error(f"Error accessing item in '{current_folder_path}': {item_error}", exc_info=True)
                    continue

        except Exception as folder_items_error:
            logging.error(f"Error accessing items in folder '{current_folder_path}': {folder_items_error}", exc_info=True)
            # If we can't access items, we count all items (if known) as failed for this folder
            error_count += item_count if 'item_count' in locals() else 0

        # Update folder summary for the source folder
        self.migration_report['folder_summary'][current_folder_path] = {
            'successful_moves_from_this_folder': success_count,
            'failed_moves_from_this_folder': error_count,
            'total_items_in_source_folder': success_count + error_count # This reflects items *attempted* from this folder
        }

        # Process subfolders, *but still move their items to the same target_inbox_folder*
        for subfolder in source_folder.Folders:
            try:
                # Pass the *original* target_inbox_folder to subfolders
                s, e = self.process_folder(subfolder, target_inbox_folder, current_folder_path)
                success_count += s
                error_count += e

            except Exception as subfolder_error:
                logging.error(f"Error processing subfolder '{subfolder.Name}': {subfolder_error}", exc_info=True)
                error_count += 1 # Count the subfolder itself as a failure if it can't be processed

        return success_count, error_count

    def generate_report(self):
        """Generate a comprehensive migration report"""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        self.migration_report['end_time'] = datetime.now().isoformat()
        if self.migration_report['start_time']:
            self.migration_report['duration_seconds'] = (
                datetime.fromisoformat(self.migration_report['end_time']) -
                datetime.fromisoformat(self.migration_report['start_time'])
            ).total_seconds()
        else:
            self.migration_report['duration_seconds'] = 0

        self.migration_report['success_rate'] = (
            self.migration_report['total_successful'] / self.migration_report['total_attempted'] * 100
        ) if self.migration_report['total_attempted'] > 0 else 0

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        # Print summary to console
        print(f"\n=== MIGRATION SUMMARY ===")
        print(f"Start Time: {self.migration_report['start_time']}")
        print(f"End Time: {self.migration_report['end_time']}")
        print(f"Duration: {self.migration_report['duration_seconds']:.2f} seconds")
        print(f"Total Items Attempted: {self.migration_report['total_attempted']}")
        print(f"Successful Moves: {self.migration_report['total_successful']}")
        print(f"Failed Moves: {self.migration_report['total_failed']}")
        print(f"Success Rate: {self.migration_report['success_rate']:.2f}%")

        if self.migration_report['failed_items']:
            print(f"\n=== FAILED ITEMS ({len(self.migration_report['failed_items'])}) ===")
            print("Showing first 5 failures (check log file for all details):")
            for i, failed_item in enumerate(self.migration_report['failed_items'][:5], 1):
                print(f"{i}. Subject: {failed_item['subject'][:70]}...")
                print(f"   From Folder: {failed_item['folder']}")
                print(f"   Error: {failed_item['error']}")
                print("-" * 30)

    def select_pst_store(self, namespace):
        """Prompts the user to select an open PST store."""
        pst_stores = []
        for store in namespace.Stores:
            # Check for PST files based on StoreType or FilePath if available
            # olStorePST = 0 in Outlook VBA, but StoreID is more reliable for identifying PSTs
            # Store.ExchangeStoreType can also distinguish OST (3) from PST (0 for non-exchange PSTs)
            if store.DisplayName.lower().endswith('.pst') or (
                hasattr(store, 'FilePath') and store.FilePath and store.FilePath.lower().endswith('.pst')
            ):
                pst_stores.append(store)

        if not pst_stores:
            logging.error("No PST files found currently open in Outlook.")
            return None

        print("\n--- Select PST File for Migration ---")
        for i, store in enumerate(pst_stores):
            print(f"{i + 1}. {store.DisplayName} (Path: {getattr(store, 'FilePath', 'N/A')})")

        while True:
            try:
                choice = input("Enter the number of the PST file to migrate from: ")
                selected_index = int(choice) - 1
                if 0 <= selected_index < len(pst_stores):
                    return pst_stores[selected_index]
                else:
                    print("Invalid choice. Please enter a number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def select_target_inbox(self, namespace):
        """Prompts the user to select the target Inbox for an OST-linked account."""
        accounts = []
        for account in namespace.Accounts:
            # Check if account is Exchange, Office 365, or IMAP (typically uses OST)
            # Account.ExchangeConnectionMode will be olOnline, olCachedExchange, etc.
            # olExchange = 0, olImap = 5
            if account.AccountType == 0 or account.AccountType == 5: # olExchange or olImap
                accounts.append(account)

        if not accounts:
            logging.error("No Exchange, Office 365, or IMAP accounts found in Outlook.")
            return None

        print("\n--- Select Target Inbox (OST-linked Account) ---")
        for i, account in enumerate(accounts):
            # Try to get the Inbox folder for the account's default store
            try:
                store = account.DeliveryStore
                root_folder = store.GetRootFolder()
                inbox = root_folder.Folders.Item("Inbox")
                print(f"{i + 1}. {account.DisplayName} (Inbox: {inbox.FolderPath})")
            except Exception as e:
                print(f"{i + 1}. {account.DisplayName} (Error accessing Inbox: {e})")
                accounts[i] = None # Mark as unselectable

        selectable_accounts = [acc for acc in accounts if acc is not None]
        if not selectable_accounts:
            logging.error("Could not find any selectable Inboxes for OST-linked accounts.")
            return None

        while True:
            try:
                choice = input("Enter the number of the account whose Inbox you want to use as the target: ")
                selected_index = int(choice) - 1
                if 0 <= selected_index < len(accounts) and accounts[selected_index] is not None:
                    selected_account = accounts[selected_index]
                    target_store = selected_account.DeliveryStore
                    target_inbox = target_store.GetRootFolder().Folders.Item("Inbox")
                    logging.info(f"Target Inbox selected: {target_inbox.FolderPath}")
                    return target_inbox
                else:
                    print("Invalid choice. Please enter a valid number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except Exception as e:
                print(f"Error getting Inbox for selected account: {e}. Please try again.")

    def run_migration(self):
        """Main migration function with comprehensive validation"""
        logging.info("Starting PST to OST migration with validation (all emails to Inbox).")
        self.migration_report['start_time'] = datetime.now().isoformat()

        pythoncom.CoInitialize()
        outlook = None
        try:
            outlook = win32com.client.Dispatch("Outlook.Application")
            namespace = outlook.GetNamespace("MAPI")

            # User selects PST store
            pst_store = self.select_pst_store(namespace)
            if not pst_store:
                logging.error("PST store selection failed. Exiting migration.")
                return False
            logging.info(f"Selected PST: {pst_store.DisplayName} (Path: {getattr(pst_store, 'FilePath', 'N/A')})")

            # User selects target Inbox
            target_inbox = self.select_target_inbox(namespace)
            if not target_inbox:
                logging.error("Target Inbox selection failed. Exiting migration.")
                return False
            logging.info(f"Selected Target Inbox: {target_inbox.FolderPath}")

            print("\n" + "=" * 50)
            print(f"CONFIRMATION: You are about to MOVE ALL emails from '{pst_store.DisplayName}'")
            print(f"into the INBOX of '{target_inbox.Parent.DisplayName}' ('{target_inbox.FolderPath}').")
            print("This action will flatten the PST folder structure into the single target Inbox.")
            print("THIS ACTION CANNOT BE UNDONE. ENSURE YOU HAVE A PST BACKUP.")
            print("=" * 50)

            confirm = input("Type 'CONFIRM' to proceed with the migration: ")
            if confirm.strip().upper() != 'CONFIRM':
                logging.warning("Migration cancelled by user.")
                print("Operation cancelled.")
                return False

            # Process all folders in the selected PST, moving items to the target_inbox
            logging.info(f"Beginning to process folders from '{pst_store.DisplayName}'.")

            # Start processing from the root folder of the PST
            success_count, error_count = self.process_folder(
                pst_store.GetRootFolder(),
                target_inbox
            )

            # Generate final report
            self.generate_report()

            if error_count == 0:
                logging.info("Migration completed successfully with 0 errors!")
            else:
                logging.warning(f"Migration completed with {error_count} errors. Please check the log and report files.")

            return error_count == 0

        except Exception as e:
            logging.error(f"Critical error during migration: {e}", exc_info=True)
            return False
        finally:
            # Release COM objects
            if outlook:
                del outlook
            pythoncom.CoUninitialize()

# Usage
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("      PST to OST Email Migration Tool (Flatten to Inbox)     ")
    print("=" * 60)
    print("This tool will MOVE emails from a selected PST file")
    print("to the INBOX of an Outlook account (typically OST-linked).")
    print("All emails will be moved to the TARGET INBOX, flattening")
    print("the original folder structure of the PST.")
    print("\n" + "=" * 60)

    migrator = PSTtoOSTMigrator()
    success = migrator.run_migration()

    if success:
        print("\n✅ Migration completed successfully!")
    else:
        print("\n❌ Migration completed with errors. Check 'migration_logs' and 'migration_reports' folders for details.")